In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import tensorflow as tf
from keras.layers import Dense, Input, Conv2D, BatchNormalization
from keras.layers import MaxPool2D, MaxPooling2D, Reshape, Dropout, SeparableConv2D
from keras.models import Model
from keras.utils import to_categorical

# matplotlib for displaying the output
import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')
%matplotlib inline
from WavDataLoader import WavDataLoader

Using TensorFlow backend.


In [3]:
labels = ['silence', 'yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']
data_dir = r'/home/shaur141/Development/kaggle/tensorflow-speech-recognition-challenge/data/train/audio'
wav_data_loader = WavDataLoader(data_dir, labels, nx=128, ny=32)


In [4]:
def conv_net(x_dict, n_classes, dropout, reuse, is_training):
    with tf.variable_scope('ConvNet', reuse=reuse):
        x = x_dict['x']
        
        x = tf.reshape(x, shape=[-1, 128, 32, 1])
        
        conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
        
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)

        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)
        
        fc1 = tf.contrib.layers.flatten(conv2)
        
        fc1 = tf.layers.dense(fc1, 1024)
        
        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)
        
        out = tf.layers.dense(fc1, n_classes)

    return out

In [5]:
num_classes = wav_data_loader.num_labels
dropout = 0.25

In [6]:
def model_fn(features, labels, mode):
    logits_train = conv_net(features, num_classes, dropout, reuse=False,
                            is_training=True)
    logits_test = conv_net(features, num_classes, dropout, reuse=True,
                           is_training=False)
    
    pred_classes = tf.argmax(logits_test, axis=1)
    pred_probs = tf.nn.softmax(logits_test)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)
        
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))

    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    
    train_op = optimizer.minimize(loss_op,
                                  global_step=tf.train.get_global_step())

    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    estim_specs = tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=pred_classes,
        loss=loss_op,
        train_op=train_op,
        eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [7]:
# model = tf.estimator.Estimator(model_fn)

In [8]:
# batch_size = 32
# num_steps = 10
# learning_rate = 0.001
# input_fn = tf.estimator.inputs.numpy_input_fn(
#     x={'x': wav_data_loader.X}, y=wav_data_loader.y,
#     batch_size=batch_size, num_epochs=None, shuffle=True)

# model.train(input_fn, steps=num_steps)

In [10]:
def build_model():
    inputs = Input(shape=(wav_data_loader.nx, wav_data_loader.ny,1))
#     x = Reshape((wav_data_loader.nx*wav_data_loader.ny,))(inputs)
#     x = BatchNormalization()(inputs)
    x = Conv2D(16,(3,3),strides=(1,1), activation='relu')(inputs)
#     x = BatchNormalization()(x)
    x = MaxPool2D(strides=(1,1))(x)
    x = Dropout(0.25)(x)
    x = Conv2D(32,(3,3),strides=(2,2), activation='relu')(x)
#     x = BatchNormalization()(x)
    x = MaxPool2D(strides=(1,1))(x)
    x = Dropout(0.25)(x)
    x = SeparableConv2D(64,(3,3),strides=(2,2), activation='relu')(x) 
#     x = BatchNormalization()(x)
    x = MaxPool2D(strides=(1,1))(x)       
    x = Reshape((-1,))(x)
    x = Dense(512, activation='relu')(x)
#     x = Dense(128, activation='relu')(x)
    predictions = Dense(wav_data_loader.num_labels, activation='softmax')(x)    
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='Nadam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model    

In [11]:
model = build_model()

In [12]:
model.fit(x=wav_data_loader.X, y=to_categorical(wav_data_loader.y), validation_split=0.15, epochs=20)

Train on 21941 samples, validate on 3873 samples
Epoch 1/20
21941/21941 [==============================] - 17s - loss: 1.2544 - acc: 0.5675 - val_loss: 0.7302 - val_acc: 0.7596
Epoch 2/20
21941/21941 [==============================] - 17s - loss: 0.5842 - acc: 0.8011 - val_loss: 0.5238 - val_acc: 0.8239
Epoch 3/20
21941/21941 [==============================] - 17s - loss: 0.4209 - acc: 0.8567 - val_loss: 0.4828 - val_acc: 0.8399
Epoch 4/20
21941/21941 [==============================] - 17s - loss: 0.3462 - acc: 0.8828 - val_loss: 0.3429 - val_acc: 0.8892
Epoch 5/20
21941/21941 [==============================] - 17s - loss: 0.2838 - acc: 0.9030 - val_loss: 0.3334 - val_acc: 0.8928
Epoch 6/20
21941/21941 [==============================] - 17s - loss: 0.2478 - acc: 0.9137 - val_loss: 0.3189 - val_acc: 0.9009
Epoch 7/20
21941/21941 [==============================] - 17s - loss: 0.2163 - acc: 0.9253 - val_loss: 0.3699 - val_acc: 0.8874
Epoch 8/20
21941/21941 [==============================]

In [13]:
model.save('model.h5')